In [ ]:
import h5py
import numpy as np
import tqdm

In [ ]:
DATASET_PATH = "dataset.h5"
INTRINSICS = {"focalLength": 1.0, "width": 384.0, "height": 256.0, "zRange": [0.01, 2.0]}
VIEW_IDX = 3

In [ ]:
with h5py.File(DATASET_PATH, "r") as f:
    dp_key = next(iter(f.keys()))
    for k, v in f[dp_key].items():
        print(f"{k:<12} {v.shape}")

In [ ]:
with h5py.File(DATASET_PATH, "r") as f:
    depths = np.concatenate([f[key]["depths"][VIEW_IDX][()] for key in f.keys()])

In [ ]:
depth_min = depths.min()
depth_max = depths.max()

print(f"depth: [{depth_min:.4f}, {depth_max:.4f}], mean={depths.mean():.4f}")

In [ ]:
with h5py.File(DATASET_PATH, "r") as f_in, h5py.File("dataset-offset.h5", "w") as f_out:
    f_out.attrs.update(f_in.attrs)
    f_out.attrs["depth_min"] = depth_min
    f_out.attrs["depth_max"] = depth_max

    for key in tqdm.tqdm(f_in.keys()):
        dp = f_in[key]

        obj_ids = dp["obj_ids"][()]
        seg = dp["seg_ids"][VIEW_IDX]
        poses = dp["poses"][()]
        target_poses = dp["target_poses"][()]
        feasibles = dp["feasibles"][()]

        depth_raw = dp["depths"][VIEW_IDX][()]
        depth = (depth_raw - depth_min) / (depth_max - depth_min)

        target_offsets = target_poses[:, :3] - poses[:, :3]

        for i in range(len(obj_ids)):
            grp = f_out.create_group(f"{key}_obj_{i}")
            grp.create_dataset("depth", data=depth)
            grp.create_dataset("mask", data=(seg == obj_ids[i]).astype(np.float32))
            grp.create_dataset("quat", data=poses[i][3:])
            grp.create_dataset("target_offset", data=target_offsets[i])
            grp.create_dataset("feasibles", data=feasibles[i])